In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [23]:
df = pd.read_csv('company_data.csv')
df

,DAY,SIR,RSI,BF,ADV,PUBLIC
0,1,0.293764,50.787689,0.02262,49272.088296,0.509402
